In [ ]:
# union all 1x2_odds files together and join to master data


In [2]:
# imports
import pandas as pd

In [3]:
# get 1x2 odds data for last few years
games_0 = pd.read_excel('data/2023_1x2_data.xlsx', header=0)
games_1 = pd.read_excel('data/2024_1x2_data.xlsx', header=0)
games_2 = pd.read_excel('data/2025_1x2_data.xlsx', header=0)

# union together
all_1x2_odds = pd.concat([games_0, games_1, games_2], ignore_index=True)

# reset index
all_1x2_odds.reset_index(drop=True, inplace=True)

# only keep key cols
key_cols = ['Game_ID'] + [x for x in all_1x2_odds.columns if x.startswith('Odds')]
all_1x2_odds = all_1x2_odds[key_cols]

# inspect
all_1x2_odds.info()
all_1x2_odds.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3349 entries, 0 to 3348
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Game_ID  3349 non-null   object 
 1   Odds_1   3224 non-null   float64
 2   Odds_X   3224 non-null   float64
 3   Odds_2   3224 non-null   float64
dtypes: float64(3), object(1)
memory usage: 104.8+ KB


,Game_ID,Odds_1,Odds_X,Odds_2
0,2023-06-24-20:00|Florida Panthers vs Edmonton ...,2.45,4.0,2.45
1,2023-06-21-20:00|Edmonton Oilers vs Florida Pa...,2.35,4.0,2.55
2,2023-06-18-20:00|Florida Panthers vs Edmonton ...,2.00,4.0,3.10
3,2023-06-15-20:00|Edmonton Oilers vs Florida Pa...,2.30,4.0,2.55
4,2023-06-13-20:00|Edmonton Oilers vs Florida Pa...,2.15,4.0,2.80


In [ ]:
# read in master games data
all_games = pd.read_excel(r'data/master_games_data.xlsx', header=0)

# inspect
all_games.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3349 entries, 0 to 3348
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Game_Link                 3349 non-null   object        
 1   Extra_Time                3349 non-null   object        
 2   Home_Team                 3349 non-null   object        
 3   Away_Team                 3349 non-null   object        
 4   Home_Score                3349 non-null   int64         
 5   Away_Score                3349 non-null   int64         
 6   P1_Home_Score             3349 non-null   int64         
 7   P1_Away_Score             3349 non-null   int64         
 8   P2_Home_Score             3349 non-null   int64         
 9   P2_Away_Score             3349 non-null   int64         
 10  P3_Home_Score             3349 non-null   int64         
 11  P3_Away_Score             3349 non-null   int64         
 12  P4_Home_Score       

,Game_Link,Extra_Time,Home_Team,Away_Team,Home_Score,Away_Score,P1_Home_Score,P1_Away_Score,P2_Home_Score,P2_Away_Score,...,Away_Conf,Home_Div,Away_Div,Conf_Matchup,Div_Matchup,Conf_Pair,Div_Pair,Team_Pair,Game_Start_Hour,Start_Hour_Group
0,https://www.flashscore.com/match/hockey/edmont...,Reg,Florida Panthers,Edmonton Oilers,2,1,1,1,1,0,...,Western,Atlantic,Pacific,False,False,Eastern-Western,Atlantic-Pacific,Florida Panthers vs Edmonton Oilers,20,Late
1,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,5,1,1,0,2,0,...,Eastern,Pacific,Atlantic,False,False,Western-Eastern,Pacific-Atlantic,Edmonton Oilers vs Florida Panthers,20,Late
2,https://www.flashscore.com/match/hockey/edmont...,Reg,Florida Panthers,Edmonton Oilers,3,5,0,1,2,3,...,Western,Atlantic,Pacific,False,False,Eastern-Western,Atlantic-Pacific,Florida Panthers vs Edmonton Oilers,20,Late
3,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,8,1,3,1,3,0,...,Eastern,Pacific,Atlantic,False,False,Western-Eastern,Pacific-Atlantic,Edmonton Oilers vs Florida Panthers,20,Late
4,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,3,4,0,1,1,3,...,Eastern,Pacific,Atlantic,False,False,Western-Eastern,Pacific-Atlantic,Edmonton Oilers vs Florida Panthers,20,Late


In [ ]:
# join the 2 dfs together
master_data = pd.merge(all_games, all_1x2_odds, how='left', on='Game_ID')

# inspect
master_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3349 entries, 0 to 3348
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Game_Link                 3349 non-null   object        
 1   Extra_Time                3349 non-null   object        
 2   Home_Team                 3349 non-null   object        
 3   Away_Team                 3349 non-null   object        
 4   Home_Score                3349 non-null   int64         
 5   Away_Score                3349 non-null   int64         
 6   P1_Home_Score             3349 non-null   int64         
 7   P1_Away_Score             3349 non-null   int64         
 8   P2_Home_Score             3349 non-null   int64         
 9   P2_Away_Score             3349 non-null   int64         
 10  P3_Home_Score             3349 non-null   int64         
 11  P3_Away_Score             3349 non-null   int64         
 12  P4_Home_Score       

,Game_Link,Extra_Time,Home_Team,Away_Team,Home_Score,Away_Score,P1_Home_Score,P1_Away_Score,P2_Home_Score,P2_Away_Score,...,Conf_Matchup,Div_Matchup,Conf_Pair,Div_Pair,Team_Pair,Game_Start_Hour,Start_Hour_Group,Odds_1,Odds_X,Odds_2
0,https://www.flashscore.com/match/hockey/edmont...,Reg,Florida Panthers,Edmonton Oilers,2,1,1,1,1,0,...,False,False,Eastern-Western,Atlantic-Pacific,Florida Panthers vs Edmonton Oilers,20,Late,2.45,4.0,2.45
1,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,5,1,1,0,2,0,...,False,False,Western-Eastern,Pacific-Atlantic,Edmonton Oilers vs Florida Panthers,20,Late,2.35,4.0,2.55
2,https://www.flashscore.com/match/hockey/edmont...,Reg,Florida Panthers,Edmonton Oilers,3,5,0,1,2,3,...,False,False,Eastern-Western,Atlantic-Pacific,Florida Panthers vs Edmonton Oilers,20,Late,2.00,4.0,3.10
3,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,8,1,3,1,3,0,...,False,False,Western-Eastern,Pacific-Atlantic,Edmonton Oilers vs Florida Panthers,20,Late,2.30,4.0,2.55
4,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,3,4,0,1,1,3,...,False,False,Western-Eastern,Pacific-Atlantic,Edmonton Oilers vs Florida Panthers,20,Late,2.15,4.0,2.80


In [7]:
# function that takes the df, sorts it by the target grain, then performs the groupby
def grain_sort_cumsum_props(home_vs_away: str, df: pd.DataFrame) -> pd.DataFrame:
    # add home or away team to sort grain
    sort_grain = [home_vs_away] + ['Date', 'Game_ID'] 

    # sort
    df = df.sort_values(by=sort_grain)

    # hard code cumsum cols
    cum_cols = ["Reg_Home_Win", "Reg_Away_Win", "Reg_Tie", 'P1_Home_Score', 'P1_Away_Score', 
                'P2_Home_Score', 'P2_Away_Score', 'P3_Home_Score', 'P3_Away_Score',
                'regular_time_score_home', 'regular_time_score_away']
    
    # add count of games for home or away
    df[f'cum_{home_vs_away}_Games'] = df.groupby(home_vs_away).cumcount() + 1

    # compute cumsum for each group
    df[[f"cum_{col}" for col in cum_cols]] = df.groupby(home_vs_away)[cum_cols].cumsum()

    # calc prop home reg win
    df['prop_Reg_Home_Win'] = df['cum_Reg_Home_Win'] / df[f'cum_{home_vs_away}_Games']
    df['prop_Reg_Away_Win'] = df['cum_Reg_Away_Win'] / df[f'cum_{home_vs_away}_Games']
    df['prop_Reg_Tie'] = df['cum_Reg_Tie'] / df[f'cum_{home_vs_away}_Games']

    # calc props for what % of goals scored in each period
    df['prop_P1_Home_Score'] = df['cum_P1_Home_Score'] / df['cum_regular_time_score_home']
    df['prop_P2_Home_Score'] = df['cum_P2_Home_Score'] / df['cum_regular_time_score_home']
    df['prop_P3_Home_Score'] = df['cum_P3_Home_Score'] / df['cum_regular_time_score_home']

    df['prop_P1_Away_Score'] = df['cum_P1_Away_Score'] / df['cum_regular_time_score_away']
    df['prop_P2_Away_Score'] = df['cum_P2_Away_Score'] / df['cum_regular_time_score_away']
    df['prop_P3_Away_Score'] = df['cum_P3_Away_Score'] / df['cum_regular_time_score_away']

    return df

# test func
test_all_home_games = grain_sort_cumsum_props('Home_Team', master_data)

# isolate cumcols
sel_cum_cols = [x for x in test_all_home_games.columns if x.startswith('cum_') or x.startswith('prop_')]

# add date, gid, and home team to list
peek_cols = ['Date', 'Game_ID', 'Home_Team'] + sel_cum_cols

# inspect cum cols
test_all_home_games[peek_cols].head(5)

,Date,Game_ID,Home_Team,cum_Home_Team_Games,cum_Reg_Home_Win,cum_Reg_Away_Win,cum_Reg_Tie,cum_P1_Home_Score,cum_P1_Away_Score,cum_P2_Home_Score,...,cum_regular_time_score_away,prop_Reg_Home_Win,prop_Reg_Away_Win,prop_Reg_Tie,prop_P1_Home_Score,prop_P2_Home_Score,prop_P3_Home_Score,prop_P1_Away_Score,prop_P2_Away_Score,prop_P3_Away_Score
812,2023-01-03,2023-01-03-21:00|Anaheim Ducks vs Toronto Mapl...,Anaheim Ducks,1,0,0,1,0,0,1,...,1,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
796,2023-01-05,2023-01-05-22:00|Anaheim Ducks vs Winnipeg Jets,Anaheim Ducks,2,0,1,1,1,0,1,...,4,0.0,0.500000,0.500000,0.500000,0.500000,0.000000,0.000000,0.250000,0.750000
780,2023-01-07,2023-01-07-20:00|Anaheim Ducks vs Detroit Red ...,Anaheim Ducks,3,0,2,1,2,1,1,...,7,0.0,0.666667,0.333333,0.500000,0.250000,0.250000,0.142857,0.285714,0.571429
678,2023-01-21,2023-01-21-20:30|Anaheim Ducks vs New York Ran...,Anaheim Ducks,4,0,3,1,4,1,1,...,12,0.0,0.750000,0.250000,0.666667,0.166667,0.166667,0.083333,0.250000,0.666667
663,2023-01-23,2023-01-23-22:00|Anaheim Ducks vs Buffalo Sabres,Anaheim Ducks,5,1,3,1,5,1,3,...,14,0.2,0.600000,0.200000,0.500000,0.300000,0.200000,0.071429,0.214286,0.714286


In [ ]:
# TODO - think about what features could be indicative/relevent for identifying matchups that would require some free hockey to settle things

# the team's conf, div, and/or overall rank; in order to calc these ranks need to calc the number of wins at that grain; another way to think about it is total goals scored vs allowed

In [ ]:
# TODO - create cumulative teams table where each row is at the team + date level and I get cumulative:
# games played, home wins/losses, away wins/losses, conf wins/losses, division wins/losses, conf rank, div rank
